In [1]:

from translate import *
# process_file("")
import glob
import os
file_names = glob.glob("data/*.csv")
file_name = file_names[1]

# process_file(file_name, api_key = os.environ['ANTHROPIC_API_KEY'])

In [2]:
df = pd.read_csv(file_name)
transcript_thai = df["Transcript"]
transcript_english = df['English Translation']
full_transcript = ('\n').join(list(transcript_english))
parse_qa_anthropic(full_transcript, api_key = os.environ['ANTHROPIC_API_KEY'], questions=qa_questions)

[]

In [4]:
thai_text = full_transcript
api_key = os.environ['ANTHROPIC_API_KEY']
questions = qa_questions

tools = []
tool_name = "qa_tool"
tool_description = qa_tool_description
# questions = ["What is the name of the person?", "What is the name of the place?", "What is the name of the thing?", "What is the name of the action?", "What is the name of the time?"]
tool = construct_qa_tool_prompt(tool_name, tool_description, questions)
tools.append(tool)

client = anthropic.Anthropic(api_key = api_key)
qa_message = {
    "role": "user",
    "content": "Parse out the questions and answers according to the specific genre and description. Here is the text: " + thai_text
}
response = client.beta.tools.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=512,
    tools=tools,
    messages=[qa_message],
)

In [5]:
response

ToolsBetaMessage(id='msg_016jgnCg9tye4Xg2yN32rEFG', content=[TextBlock(text="<thinking>\nThe relevant tool to extract question/answer pairs from the given text is the qa_tool. To use it, I need to go through each parameter and see if a value can be provided based on the input text.\n\nFor the question_X_present parameters, I need to check if the text contains questions similar to the reference questions. After reviewing the text, I don't see any questions that closely match the reference questions about insurance denials, drug use, medical diagnoses, tests/treatments in the past few years, or current health symptoms. So I would set all the question_X_present parameters to false.\n\nSince no relevant questions are present, the question_X and answer_X parameters can be left blank.\n\nIn summary, I don't have enough information in the input text to populate the required parameters for the qa_tool. The text appears to be a casual conversation that doesn't contain the type of structured hea

In [15]:
def parse_qa_anthropic(thai_text, questions, api_key):
    tools = []
    tool_name = "qa_tool"
    tool_description = qa_tool_description
    # questions = ["What is the name of the person?", "What is the name of the place?", "What is the name of the thing?", "What is the name of the action?", "What is the name of the time?"]
    tool = construct_qa_tool_prompt(tool_name, tool_description, questions)
    tools.append(tool)

    client = anthropic.Anthropic(api_key = api_key)
    qa_message = {
        "role": "user",
        "content": "Parse out the questions and answers according to the specific genre and description. Here is the text: " + thai_text
    }
    response = client.beta.tools.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=512,
        tools=tools,
        messages=[qa_message],
    )
    calls = parse_qa_calls(response, n_types = len(questions))
    return calls